In [1]:
from trafficgraphnn.sumo_network import SumoNetwork
from trafficgraphnn.utils import parse_detector_output_xml, parse_tls_output_xml
from lxml import etree
import pandas as pd
import sumolib
from sumolib.net import readNet
import networkx as nx
import six

2018-07-26 14:42:41,411 matplotlib.font_manager:1465 DEBUG    Using fontManager instance from /home/simon/.cache/matplotlib/fontList.json
2018-07-26 14:42:41,695 matplotlib.backends:90  DEBUG    backend module://ipykernel.pylab.backend_inline version unknown
2018-07-26 14:42:41,835 matplotlib.backends:90  DEBUG    backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
# setup: create an object that represents all information associated with a Sumo road network and its output data
#sn = SumoNetwork(
#    'data/networks/testgrid2/testgrid2.net.xml', routefile='data/networks/testgrid2/testgrid2_rand_routes.routes.xml',
#    lanewise=True, addlfiles=[
#        'data/networks/testgrid2/testgrid2_e2.add.xml',
#        'data/networks/testgrid2/testgrid2_e1.add.xml',
#        'data/networks/testgrid2/tls_output.add.xml']
#)

sn = SumoNetwork(
    'data/networks/simonnet/simonnet.net.xml', routefile='data/networks/simonnet/simonnet_rand_routes.routes.xml',
    lanewise=True, addlfiles=['data/networks/simonnet/simonnet_e1.add.xml', 'data/networks/simonnet/simonnet_e2.add.xml', 'data/networks/simonnet/tls_output.add.xml']
)

In [3]:
#sn.load_data_to_graph()

In [4]:
#sn.run()

In [5]:
graph = sn.get_graph()

In [31]:
# List of all lanes. This is a 5x5 grid-shaped network. In this naming scheme, a lane named x1/y1tox2/y2_z means
# it is the zth lane on the road that goes from the junction in the x1/y1 position of the grid to the junction
# in the x2/y2 position.
# The lanes here are called nodes (rather than edges) because traditionally
# in graphs the discrete locations at which data of interest
# exist (e.g., our queues and pressure) are thought of as the nodes.
#graph.edges[('0/0to0/1_0', '0/1to1/1_0')]
#graph.get_edge_data('0/0to0/1_0', '0/1to1/1_0')

#graph.nodes['0/0tobottom0_0']


for lane in graph.nodes(data=True):
    print(lane[1])
    print('----------------')
    
    



{'detectors': {'e1_0-0to0-1_0_0': {'freq': '1', 'friendlyPos': 'x', 'id': 'e1_0-0to0-1_0_0', 'file': 'output/simonnet_e1_0-0to0-1_0_0_output.xml', 'lane': '0/0to0/1_0', 'pos': '572.3', 'type': 'e1Detector'}, 'e1_0-0to0-1_0_1': {'freq': '1', 'friendlyPos': 'x', 'id': 'e1_0-0to0-1_0_1', 'file': 'output/simonnet_e1_0-0to0-1_0_1_output.xml', 'lane': '0/0to0/1_0', 'pos': '452.29999999999995', 'type': 'e1Detector'}, 'e2_0-0to0-1_0_0': {'freq': '1', 'friendlyPos': 'x', 'id': 'e2_0-0to0-1_0_0', 'file': 'output/simonnet_e2_0-0to0-1_0_0_output.xml', 'lane': '0/0to0/1_0', 'length': '577.3', 'pos': '0', 'type': 'e2Detector'}}}
----------------
{'detectors': {'e1_0-0to0-1_1_0': {'freq': '1', 'friendlyPos': 'x', 'id': 'e1_0-0to0-1_1_0', 'file': 'output/simonnet_e1_0-0to0-1_1_0_output.xml', 'lane': '0/0to0/1_1', 'pos': '572.3', 'type': 'e1Detector'}, 'e1_0-0to0-1_1_1': {'freq': '1', 'friendlyPos': 'x', 'id': 'e1_0-0to0-1_1_1', 'file': 'output/simonnet_e1_0-0to0-1_1_1_output.xml', 'lane': '0/0to0/1_1'

In [ ]:
# For illustration purposes, pick a specific lane
lane = '0/0to0/1_0'

In [ ]:
# This python dict of dicts contains info about each detector on this lane.
# There are two types of detectors mentioned here. e1 detectors are simulated loop detectors.
# e2 detectors are "lane area" detectors that we use to collect ground-truth queue length readings

graph.node[lane]['detectors']

In [ ]:
# Each key in this dict corresponds to a detector.
graph.node[lane]['detectors'].keys()
graph.get_edge_data()

In [ ]:
# Querying the information about this particular detector, we see its configuration info
# (position, reporting frequency (here, 60s), etc.) as well as a pandas dataframe for its data
graph.node[lane]['detectors']['e2_0-0to0-1_0_0']

In [ ]:
# The pandas dataframe above was formed by selecting from this larger dataframe that holds all e1 records
all_e1_data = sn.data_dfs[1]

In [ ]:
all_e1_data.head()

In [ ]:
# An easy way to get all data for a particular lane is by selecting from this larger dataframe

In [ ]:
lane_data = all_e1_data.xs(lane, level='node_id')

In [ ]:
lane_data.head()

In [ ]:
all_e2_data = sn.data_dfs[0]

In [ ]:
# Edge data: Edges represent connections between two nodes (lanes).
# We can all edges:
graph.edges

In [ ]:
# Edges that originate from a lane:
graph.edges(lane)

In [ ]:
# Pick a specific edge to see its data:
lane_out = '0/1to0/2_0'
graph.edges[(lane, lane_out)]

In [ ]:
graph.edges[(lane, lane_out)]['switch_times']

In [ ]:
# This data df is the big one that contains all edge traffic light switch timings.
# Right now the only way to get it is to select the last element of this list
tls_df = sn.data_dfs[-1]

In [ ]:
tls_df.head()

In [ ]:
tls_df.xs('1/2to1/1_0', level='fromLane')

The "graph" object we are using is an instance of the DiGraph class from the package "networkx". For more documentation on how to work with the nodes and edges in this object, see the documentation for networkx: https://networkx.github.io/documentation/stable/.